###**Problem Description :-**

Harry and Hermione are in the 2nd year of Hogwarts (for those who haven't watched Harry Potter series, it's a magic school). While wandering around Hogwarts, they came across a room filled with new potions that are being created. Both Harry and Hermione want to find out the category that each of the potions belong to, so that they can use them for appropriate scenarios.

Assume that there are only 4 categories of potions that are being prepared in the room. The compositions of each potion vary over a range of values. A book with the data of various potions, their compositions and the corresponding categories is available in the school library.

Categories of the potions are: anti-aging, transmigration, memory and healing.
Help the wizards identify the types of potions and support them in perfecting the art of magic!

**Training Model :-**

In [1]:
from sklearn.metrics import f1_score
import numpy as np
import csv

In [2]:
X = np.genfromtxt("train_X_lg_v2.csv", delimiter=',', dtype=np.float64, skip_header=1)
Y = np.genfromtxt("train_Y_lg_v2.csv", delimiter=',', dtype=np.float64)

In [3]:
def sigmoid(Z):
    A = 1.0 / (1.0 + np.exp(-Z))
    return A

In [4]:
def compute_cost(X, Y, W, b):
    m = len(Y)
    Z = np.dot(X, W.T) + b
    A = sigmoid(Z)
    A[A == 1] = 0.9999
    A[A == 0] = 0.0001
    cost = -1 * (1/m) * np.sum(np.multiply(Y, np.log(A)) +
                               np.multiply((1 - Y), np.log(1 - A)))
    return cost

In [5]:
def compute_gradient_of_cost_function(X, Y, W, b):
    m = len(Y)
    Z = np.dot(X, W.T) + b
    A = sigmoid(Z)
    db = np.sum(A - Y) / m
    dw = np.dot((A - Y).T, X) / m
    return dw, db

In [6]:
def Optimize_weights_using_gradient_descent(X, Y, learning_rate):
    Threshold_value = 0.0000001
    prev_cost, b, i = 0, 0, 1
    Y = Y.reshape(X.shape[0], 1)
    W = np.zeros((1, X.shape[1]))
    while True:
        dw, db = compute_gradient_of_cost_function(X, Y, W, b)
        W = W - (learning_rate * dw)
        b = b - (learning_rate * db)
        cost = compute_cost(X, Y, W, b)
        if abs(cost - prev_cost) < (Threshold_value):
            break
        prev_cost = cost
        i += 1
    return W, b

In [7]:
def get_train_data_for_class(train_X, train_Y, class_label):
    class_X = np.copy(train_X)
    class_Y = np.copy(train_Y)
    class_Y = np.where(class_Y == class_label, 1, 0)
    return class_X, class_Y

In [11]:
Y_value = len(np.unique(Y))
alpha = {0:0.0055, 1:0.0032, 2:0.0028, 3:0.0061}

In [12]:
  weights = list()
  for i in range(Y_value):
      class_X, class_Y = get_train_data_for_class(X, Y, i)
      w, b = Optimize_weights_using_gradient_descent(class_X, class_Y, alpha[i])
      weights.append(np.insert(w, 0, b, axis=1))

**Storing Weights corresponding to different Y values :-**

In [13]:
for i in weights:
    print(*i)

[ 5.33142623e+00  2.81877446e-01  2.43659021e-01 -2.67472044e-01
 -2.33522021e+00  4.02551060e-01  7.98272411e-01 -1.76610869e+01
  2.27402110e-01  3.61402868e-01  3.10870816e-01  1.98711735e-01
 -4.15589694e+00  3.60830640e-01  1.03425315e-01 -3.61782139e-01
 -1.38499698e-03  3.85566879e-01  1.09210238e-01 -2.81900533e+00
  5.77059732e-01]
[-9.62556144e-02 -3.28939963e-02 -6.05460171e-02  1.60073034e-01
 -1.34642070e-01  7.70465691e-02 -5.61231661e-01 -1.68156795e+00
 -5.82736856e-02  4.06520302e-04 -4.30437757e-03  9.10515206e-02
 -2.96724656e-01  1.66495404e-01  4.12898441e-01  2.24697969e-01
  1.14281772e-03 -1.12329882e-01  2.93416874e-01  1.27637632e-01
 -1.07731868e-02]
[-0.41538396 -0.01448607 -0.00896293 -0.2168049  -0.09514797 -0.17355246
  0.06634997  1.59179279 -0.1240571  -0.10812952 -0.17353211 -0.26853718
 -0.17361844 -0.13508278 -0.05694573  0.08030582 -0.00894024 -0.1417076
 -0.22331703 -0.21229124  0.31379077]
[-1.37986218e+01 -1.05676248e-01 -1.64990468e-01  2.218591

**Prediction and Validation of model using f1 score :-**

In [14]:
def predict_target_values(test_X, weights):
    Y_value = [0, 1, 2, 3]
    b_for_all = weights[:,0][:,0]
    Weights_for_all = weights[:,0][:, 1:]
    pred_Y, list = [], []
    m, n = len(test_X), len(weights)
    for i in range(n):
        Z = np.dot(test_X, Weights_for_all[i].T) + b_for_all[i]
        list.append(sigmoid(Z))
    Predicted_arr = np.array(list)
    max_value = np.max(Predicted_arr, axis = 0)
    for idx in range(m):
        for y in Y_value:
            if(Predicted_arr[y][idx] == max_value[idx]):
                pred_Y.append(y)
                break
    predicted_Y_value = np.array(pred_Y).reshape(test_X.shape[0], 1)
    return predicted_Y_value

In [15]:
weight = np.array(weights)
w = []
for i in weight:
    w.append(i[0])

In [16]:
predicted_Y_value = predict_target_values(X, weight)
predicted_Y_value

array([[1],
       [0],
       [3],
       ...,
       [0],
       [3],
       [1]])

In [17]:

weighted_f1_score = f1_score(Y, predicted_Y_value, average = 'weighted')
weighted_f1_score

0.8222318714002458